# Introduction
This notebook implements services for Interfacer inspired by [this](https://interfacer-gui-staging.dyne.org/project/062REP7DAN6ZZCR1EJFW2JX0FC) case

# Initialisation

In [ ]:
# The following should reload files if we change them on disk
%load_ext autoreload
%aimport if_lib, if_utils, if_dpp, if_graphics, if_consts, if_prop, if_groups
%autoreload 1
import os
import json
import random
from datetime import datetime, timezone, timedelta

from if_utils import get_filename, show_data, save_traces

from if_lib import generate_random_challenge, read_HMAC, read_keypair, get_id_person, get_location_id, \
get_unit_id, get_resource_spec_id, get_resource, get_process, create_event, make_transfer, reduce_resource, \
set_user_location

from if_groups import get_processgrp, insert_procingrp

from if_prop import get_proposal, get_intent, get_proposedIntent, show_proposal, get_satisfaction, check_proposals

from if_dpp import trace_query, check_traces, er_before, get_dpp

from if_graphics import vis_dpp, make_sankey, consol_trace

In [ ]:
# We define the constant for our case
USE_CASE = 'ifservices'

# What endpoint are we talking to?
# debug
# ENDPOINT = 'http://65.109.11.42:10000/api'
ENDPOINT = 'http://zenflows-debug.interfacer.dyne.org/api'
# staging
# ENDPOINT = 'http://65.109.11.42:8000/api'
# ENDPOINT = 'https://zenflows-staging.interfacer.dyne.org/api'
# testing
# ENDPOINT = 'http://65.109.11.42:9000/api'
# ENDPOINT = 'https://zenflows-test.interfacer.dyne.org/api'
# ENDPOINT = 'http://localhost:8000/api'

USERS = ['designer1', 'designer2', 'service_prov1', 'service_prov2', 'manufacturer1', 'manufacturer2', 'customer1', 'customer2']

In [ ]:
# Calculate names of settings files
USERS_FILE = get_filename('cred_users.json', ENDPOINT, USE_CASE)
LOCS_FILE = get_filename('loc_users.json', ENDPOINT, USE_CASE)
UNITS_FILE = get_filename('units_data.json', ENDPOINT, USE_CASE)
SPECS_FILE = get_filename('res_spec_data.json', ENDPOINT, USE_CASE)

In [ ]:
# Read or define user data that is going to be used in the GraphQL calls

# create data structure to hold processes
process_data = {}

# create data structure to hold process groups
processgrp_data = {}

# create data structure to hold proposals
proposal_data = {}

# create data structure to hold intents
intent_data = {}

# create data structure to hold proposed intents
prop_int_data = {}

# create data structure to hold satisfactions
satisfaction_data = {}

# create data structures to hold resources and events (to compare/check results from track and trace)
res_data = {}
event_seq = []

if os.path.isfile(USERS_FILE):
    with open(USERS_FILE,'r') as f:
        users_data = json.loads(f.read())
    print("Credentials file available for users")
else:
    users_data = {}
    users_data['designer1'] = {
      "userChallenges": {
        "whereParentsMet": "London",
        "nameFirstPet": "Fuffy",
        "nameFirstTeacher": "Jim",
        "whereHomeTown": "Paris",
        "nameMotherMaid": "Wright"
      },
      "name": "Designer1",
      "username": "designer1_username",
      "email": "designer1@example.org",
      "note": "me.designer1.org"
    }
    users_data['designer2'] = {
      "userChallenges": {
        "whereParentsMet": "London",
        "nameFirstPet": "Fido",
        "nameFirstTeacher": "Mary",
        "whereHomeTown": "Amsterdam",
        "nameMotherMaid": "Wraight"
      },
      "name": "Designer2",
      "username": "designer2_username",
      "email": "designer2@example.org",
      "note": "me.designer2.org"
    }

    users_data['manufacturer1'] = {
        "userChallenges": {
            "whereParentsMet":"Amsterdam",
            "nameFirstPet":"Toby",
            "nameFirstTeacher":"Juliet",
            "whereHomeTown":"Rome",
            "nameMotherMaid":"Banks"
        },
        "name": "Manufacturer1",
        "username": "manufacturer1_username",
        "email": "manufacturer1@example.org",
        "note" : "me.manufacturer1.org"
    }
    users_data['manufacturer2'] = {
        "userChallenges": {
            "whereParentsMet":"Canicatti",
            "nameFirstPet":"Ula",
            "nameFirstTeacher":"Pugsly",
            "whereHomeTown":"Florence",
            "nameMotherMaid":"Ranks"
        },
        "name": "Manufacturer2",
        "username": "manufacturer2_username",
        "email": "manufacturer2@example.org",
        "note" : "me.manufacturer2.org"
    }
    users_data['service_prov1'] = {
        "userChallenges": {
            "whereParentsMet":"Amsterdam",
            "nameFirstPet":"Toby",
            "nameFirstTeacher":"Juliet",
            "whereHomeTown":"Rome",
            "nameMotherMaid":"Banks"
        },
        "name": "Service_prov1",
        "username": "service_prov1_username",
        "email": "service_prov1@example.org",
        "note" : "me.service_prov1.com"
    }
    users_data['service_prov2'] = {
        "userChallenges": {
            "whereParentsMet":"Canicatti",
            "nameFirstPet":"Ula",
            "nameFirstTeacher":"Pugsly",
            "whereHomeTown":"Florence",
            "nameMotherMaid":"Ranks"
        },
        "name": "service_prov2",
        "username": "service_prov2_username",
        "email": "service_prov2@example.org",
        "note" : "me.service_prov2.com"
    }


    users_data['customer1'] = {
        "userChallenges": {
            "whereParentsMet":"Rome",
            "nameFirstPet":"Ku",
            "nameFirstTeacher":"George",
            "whereHomeTown":"Florence",
            "nameMotherMaid":"Canti"
        },
        "name": "Customer1",
        "username": "customer1_username",
        "email": "customer1@example.org",
        "note" : "me.customer1.org"
    }

    users_data['customer2'] = {
        "userChallenges": {
            "whereParentsMet":"Rome",
            "nameFirstPet":"Ku",
            "nameFirstTeacher":"George",
            "whereHomeTown":"Florence",
            "nameMotherMaid":"Canti"
        },
        "name": "Customer2",
        "username": "customer2_username",
        "email": "customer2@example.org",
        "note" : "me.customer2.org"
    }

    with open(USERS_FILE,'w') as f:
        json.dump(users_data, f)

if os.path.isfile(LOCS_FILE):
    with open(LOCS_FILE,'r') as f:
        locs_data = json.loads(f.read())
    print("Location file available")
else:
    locs_data = {}
    locs_data['designer1'] = {
        "name": "Dyne",
        "lat": 52.39679,
        "long": 4.8781073,
        "addr": "Haparandadam 7, A1, 1013 AK Amsterdam, Netherlands",
        "note": "location.designer1.org"
    }
    locs_data['designer2'] = {
        "name": "Farback",
        "lat": 52.3767127,
        "long": 4.8990591,
        "addr": "Prins Hendrikkade 82 A, 1012 AE, Amsterdam, Netherlands",
        "note": "location.designer2.org"
    }

    locs_data['manufacturer1'] = {
        "name": "Fab Lab Hamburg",
        "lat" : 51.1531305,
        "long" : 5.2685045,
        "addr" : "Stockmeyerstr. 43 Halle 4K Eingang von der Wasserseite, 20457 Hamburg, Germany",
        "note": "location.manufacturer1.org"
    }

    locs_data['manufacturer2'] = {
        "name": "Fab Lab Amsterdam",
        "lat" : 52.372773,
        "long" : 4.8981243,
        "addr" : "Nieuwmarkt 4, 1012 CR Amsterdam, Netherlands",
        "note": "location.manufacturer2.org"
    }

    locs_data['service_prov1'] = {
        "name": "Bike Totaal B.V.",
        "lat" : 52.1971579,
        "long" : 5.3924199,
        "addr" : "Spaceshuttle 22, 3824 ML Amersfoort, Netherlands",
        "note": "location.service_prov1.com"
    }

    locs_data['service_prov2'] = {
        "name": "Lyondell Covestro Manufacturing",
        "lat" : 51.9650382,
        "long" : 4.0133683,
        "addr" : "Australiëweg 7, 3199 KB Maasvlakte Rotterdam, Netherlands",
        "note": "location.service_prov2.com"
    }

    locs_data['customer1'] = {
        "name": "CleanLease",
        "lat" : 51.47240440868687,
        "long" : 5.412460440524406,
        "addr" : "De schakel 30, 5651 Eindhoven, Netherlands",
        "note": "location.customer1.org"
    }

    locs_data['customer2'] = {
        "name": "OLVG",
        "lat" : 52.3585703,
        "long" : 4.9124307,
        "addr" : "Oosterpark 9, 1091 AC Amsterdam, Netherlands",
        "note": "location.customer2.org"
    }

    with open(LOCS_FILE,'w') as f:
        json.dump(locs_data, f)

if os.path.isfile(UNITS_FILE):
    with open(UNITS_FILE,'r') as f:
        units_data = json.loads(f.read())
    print(f"Unit file available")
else:
    units_data = {}
#     with open(file,'w') as f:
#         json.dump(units_data, f)


if os.path.isfile(SPECS_FILE):
    with open(SPECS_FILE,'r') as f:
        res_spec_data = json.loads(f.read())
    print(f"Resource Spec file available")
else:
    res_spec_data = {}

In [ ]:
# Read HMAC or get it from the server
for user in USERS:
    read_HMAC(USERS_FILE, users_data, user, endpoint=ENDPOINT)

In [ ]:
# Read the keypair
for user in USERS:
    read_keypair(USERS_FILE, users_data, user)

In [ ]:
# read or get id of the person
for user in USERS:
    get_id_person(USERS_FILE, users_data, user, endpoint=ENDPOINT)

In [ ]:
# Read of get the location id
for user in USERS:
    get_location_id(LOCS_FILE, users_data[user], locs_data, user, endpoint=ENDPOINT)
    set_user_location(USERS_FILE, users_data, locs_data, user, endpoint=ENDPOINT)

In [ ]:
# Get the ids of all units
get_unit_id(UNITS_FILE, users_data['designer1'], units_data, 'piece', 'u_piece', 'om2:one', endpoint=ENDPOINT)
get_unit_id(UNITS_FILE, users_data['designer1'], units_data, 'mass', 'kg', 'om2:kilogram', endpoint=ENDPOINT)
get_unit_id(UNITS_FILE, users_data['designer1'], units_data, 'volume', 'lt', 'om2:litre', endpoint=ENDPOINT)
get_unit_id(UNITS_FILE, users_data['designer1'], units_data, 'time', 'h', 'om2:hour', endpoint=ENDPOINT)
get_unit_id(UNITS_FILE, users_data['designer1'], units_data, 'distance', 'km', 'om2:kilometer', endpoint=ENDPOINT)

# Proposals

## These steps create the "Service" that manufactures an aluminium bike frame

In [ ]:
# Create resource specification for aluminium bike frame
name = 'aluminium_bike_frame'
note = 'Repository for aluminium bike frame'
classification = 'https://github.com/manufacturer1/aluminium_bike_frame'
default_unit_id = units_data['piece']['id']
get_resource_spec_id(SPECS_FILE, users_data['manufacturer1'], res_spec_data, name, note, classification, default_unit_id, endpoint=ENDPOINT)


In [ ]:
# Resource specification for aluminium pipes
name = 'aluminium'
note = 'Aluminium 10mt, 4cm pipes'
classification = 'https://www.wikidata.org/wiki/Q663'
default_unit_id = units_data['mass']['id']
get_resource_spec_id(SPECS_FILE, users_data['manufacturer1'], res_spec_data, name, note, classification, default_unit_id, endpoint=ENDPOINT)


In [ ]:
# Resource specification for manifacturing work
name = 'manufacturing_work'
note = 'Specification for manufacturing work'
classification = 'https://www.wikidata.org/wiki/Q187939'
default_unit_id = units_data['time']['id']
get_resource_spec_id(SPECS_FILE, users_data['manufacturer1'], res_spec_data, name, note, classification, default_unit_id, endpoint=ENDPOINT)

In [ ]:
# Create the process that manifactures the bike frame
process_name = 'Manifacture_aluminium_bike_frame'
user_data = users_data['manufacturer1']
note = f"Creation of aluminum bike frame by {user_data['name']}"
get_process(process_name, process_data, note, user_data, endpoint=ENDPOINT)

In [ ]:
# Create the proposal for encapsulating intent to manufacture bike frame
name = "Bike_frame_manifacturing_proposal"
note = "Proposal for manifacturing bike frame"
user_data = users_data['manufacturer1']

hasBeginning = datetime.now(timezone.utc).isoformat()
dur_days = 365
hasEnd = (datetime.now(timezone.utc) + timedelta(days=dur_days)).isoformat()

eligibleLocation = locs_data['manufacturer1']['id']

unitBased = True

get_proposal(name, proposal_data, note, user_data,\
             hasBeginning=hasBeginning, hasEnd=hasEnd, \
             eligibleLocation=eligibleLocation, unitBased=unitBased, endpoint=ENDPOINT)

In [ ]:
# Create the intent to consume aluminium for a bike frame

name = "Bike_frame_consume_intent"
note = "Intent for consuming aluminium for a bike frame"
user_data = users_data['manufacturer1']

# Relates an intent to a verb, such as consume, produce, work, improve, etc.
action = 'consume'
# References a concept in a common taxonomy or other classification scheme for purposes of categorization 
# or grouping.
resourceClassifiedAs = None
# The primary resource specification or definition of an existing or potential economic resource. 
# A resource will have only one, as this specifies exactly what the resource is.
resourceConformsTo = res_spec_data['aluminium']['id']
# When a specific `EconomicResource` is known which can service the `Intent`, this defines that resource.
# TODO: In this case should we specify the aluminium?
resourceInventoriedAs = None
# Since we want to produce 1 bike frame we will put one here, the correct unity will be derived from the res specs
amount = 5
# Reference to an agreement between agents which specifies the rules or policies or calculations 
# which govern this intent.
agreedIn = None
# The place where an intent would occur. Usually mappable.
atLocation = locs_data['manufacturer1']['id']
# The total quantity of the offered resource available.
# TODO: should we here specify a production capability, like 10 per day?
availableQuantity = None
# The amount and unit of the work or use or citation effort-based action. This is often a time duration, 
# but also could be cycle counts or other measures of effort or usefulness.
effortQuantity = None
# The intent is complete or not. This is irrespective of if the original goal has been met, and indicates that no more will be done.
finished = False
# The time something is expected to be complete.
due = None
# The planned beginning of the intent.
hasBeginning = datetime.now(timezone.utc).isoformat()
# The planned end of the intent.
dur_days = 365
hasEnd = (datetime.now(timezone.utc) + timedelta(days=dur_days)).isoformat()
# The planned date/time for the intent. Can be used instead of beginning and end.
hasPointInTime = None
# Defines the process to which this intent is an input.
inputOf = process_data['Manifacture_aluminium_bike_frame']['id']
# Defines the process to which this intent is an output.
outputOf = None
# The economic agent from whom the intent is initiated. This implies that the intent is an offer.
provider = users_data['manufacturer1']['id']
# The economic agent whom the intent is for. This implies that the intent is a request.
receiver = None

get_intent(name, intent_data, note, user_data, res_spec_data, provider=provider, receiver=receiver, action=action, \
           resourceClassifiedAs=resourceClassifiedAs, resourceConformsTo=resourceConformsTo, \
           resourceInventoriedAs=resourceInventoriedAs, amount=amount, agreedIn=agreedIn, \
           atLocation=atLocation, availableQuantity=availableQuantity, effortQuantity=effortQuantity,
           finished=finished, due=due, hasBeginning=hasBeginning, hasEnd=hasEnd, hasPointInTime=hasPointInTime,
           inputOf=inputOf, outputOf=outputOf, endpoint=ENDPOINT)


In [ ]:
# Create the intent to work on a bike frame

name = "Bike_frame_work_intent"
note = "Intent for working on bike frame"
user_data = users_data['manufacturer1']

# Relates an intent to a verb, such as consume, produce, work, improve, etc.
action = 'work'
# References a concept in a common taxonomy or other classification scheme for purposes of categorization 
# or grouping.
resourceClassifiedAs = None
# The primary resource specification or definition of an existing or potential economic resource. 
# A resource will have only one, as this specifies exactly what the resource is.
resourceConformsTo = res_spec_data['aluminium_bike_frame']['id']
# When a specific `EconomicResource` is known which can service the `Intent`, this defines that resource.
# TODO: In this case should we specify the aluminium?
resourceInventoriedAs = None
# Since we want to produce 1 bike frame we will put one here, the correct unity will be derived from the res specs
amount = 1
# Reference to an agreement between agents which specifies the rules or policies or calculations 
# which govern this intent.
agreedIn = None
# The place where an intent would occur. Usually mappable.
atLocation = locs_data['manufacturer1']['id']
# The total quantity of the offered resource available.
# TODO: should we here specify a production capability, like 10 per day?
availableQuantity = None
# The amount and unit of the work or use or citation effort-based action. This is often a time duration, 
# but also could be cycle counts or other measures of effort or usefulness.
effortQuantity = {}
effortQuantity['hasUnit'] = res_spec_data['manufacturing_work']['defaultUnit']
effortQuantity['hasNumericalValue'] = 8

# The intent is complete or not. This is irrespective of if the original goal has been met, and indicates that no more will be done.
finished = False
# The time something is expected to be complete.
due = None
# The planned beginning of the intent.
hasBeginning = datetime.now(timezone.utc).isoformat()
# The planned end of the intent.
dur_days = 365
hasEnd = (datetime.now(timezone.utc) + timedelta(days=dur_days)).isoformat()
# The planned date/time for the intent. Can be used instead of beginning and end.
hasPointInTime = None
# Defines the process to which this intent is an input.
inputOf = process_data['Manifacture_aluminium_bike_frame']['id']
# Defines the process to which this intent is an output.
outputOf = None
# The economic agent from whom the intent is initiated. This implies that the intent is an offer.
provider = users_data['manufacturer1']['id']
# The economic agent whom the intent is for. This implies that the intent is a request.
receiver = None

get_intent(name, intent_data, note, user_data, res_spec_data, provider=provider, receiver=receiver, action=action, \
           resourceClassifiedAs=resourceClassifiedAs, resourceConformsTo=resourceConformsTo, \
           resourceInventoriedAs=resourceInventoriedAs, amount=amount, agreedIn=agreedIn, \
           atLocation=atLocation, availableQuantity=availableQuantity, effortQuantity=effortQuantity,
           finished=finished, due=due, hasBeginning=hasBeginning, hasEnd=hasEnd, hasPointInTime=hasPointInTime,
           inputOf=inputOf, outputOf=outputOf, endpoint=ENDPOINT)


In [ ]:
# Create the intent to produce a bike frame

name = "Bike_frame_produce_intent"
note = "Intent for bike frame production"
user_data = users_data['manufacturer1']

# Relates an intent to a verb, such as consume, produce, work, improve, etc.
action = 'produce'
# References a concept in a common taxonomy or other classification scheme for purposes of categorization 
# or grouping.
resourceClassifiedAs = None
# The primary resource specification or definition of an existing or potential economic resource. 
# A resource will have only one, as this specifies exactly what the resource is.
resourceConformsTo = res_spec_data['aluminium_bike_frame']['id']
# When a specific `EconomicResource` is known which can service the `Intent`, this defines that resource.
# TODO: In this case should we specify the aluminium?
resourceInventoriedAs = None
# Since we want to produce 1 bike frame we will put one here, the correct unity will be derived from the res specs
amount = 1
# Reference to an agreement between agents which specifies the rules or policies or calculations 
# which govern this intent.
agreedIn = None
# The place where an intent would occur. Usually mappable.
atLocation = locs_data['manufacturer1']['id']
# The total quantity of the offered resource available.
# TODO: should we here specify a production capability, like 10 per day?
availableQuantity = None
# The amount and unit of the work or use or citation effort-based action. This is often a time duration, 
# but also could be cycle counts or other measures of effort or usefulness.
# TODO: for the moment we do not quantify it, but it might be important, or it might be determined when the 
# event is performed.
effortQuantity = None
# The intent is complete or not. This is irrespective of if the original goal has been met, and indicates that no more will be done.
finished = False
# The time something is expected to be complete.
due = None
# The planned beginning of the intent.
hasBeginning = datetime.now(timezone.utc).isoformat()
# The planned end of the intent.
dur_days = 365
hasEnd = (datetime.now(timezone.utc) + timedelta(days=dur_days)).isoformat()
# The planned date/time for the intent. Can be used instead of beginning and end.
hasPointInTime = None
# Defines the process to which this intent is an input.
inputOf = None
# Defines the process to which this intent is an output.
outputOf = process_data['Manifacture_aluminium_bike_frame']['id']
# The economic agent from whom the intent is initiated. This implies that the intent is an offer.
provider = users_data['manufacturer1']['id']
# The economic agent whom the intent is for. This implies that the intent is a request.
receiver = None

get_intent(name, intent_data, note, user_data, res_spec_data, provider=provider, receiver=receiver, action=action, \
           resourceClassifiedAs=resourceClassifiedAs, resourceConformsTo=resourceConformsTo, \
           resourceInventoriedAs=resourceInventoriedAs, amount=amount, agreedIn=agreedIn, \
           atLocation=atLocation, availableQuantity=availableQuantity, effortQuantity=effortQuantity,
           finished=finished, due=due, hasBeginning=hasBeginning, hasEnd=hasEnd, hasPointInTime=hasPointInTime,
           inputOf=inputOf, outputOf=outputOf, endpoint=ENDPOINT)


In [ ]:
# Create the proposed intent that publishes the intent in the proposal
name = 'Bike_frame_consume_publishing'
publishedIn = proposal_data['Bike_frame_manifacturing_proposal']['id']
publishes = intent_data['Bike_frame_consume_intent']['id']
reciprocal = False

get_proposedIntent(name, prop_int_data, user_data, publishedIn=publishedIn, publishes=publishes, \
                   reciprocal=reciprocal, endpoint=ENDPOINT)

In [ ]:
# Create the proposed intent that publishes the intent in the proposal
name = 'Bike_frame_work_publishing'
publishedIn = proposal_data['Bike_frame_manifacturing_proposal']['id']
publishes = intent_data['Bike_frame_work_intent']['id']
reciprocal = False

get_proposedIntent(name, prop_int_data, user_data, publishedIn=publishedIn, publishes=publishes, \
                   reciprocal=reciprocal, endpoint=ENDPOINT)

In [ ]:
# Create the proposed intent that publishes the intent in the proposal
name = 'Bike_frame_manifacturing_publishing'
publishedIn = proposal_data['Bike_frame_manifacturing_proposal']['id']
publishes = intent_data['Bike_frame_produce_intent']['id']
reciprocal = False

get_proposedIntent(name, prop_int_data, user_data, publishedIn=publishedIn, publishes=publishes, \
                   reciprocal=reciprocal, endpoint=ENDPOINT)

## These steps create the "Service" that assembles a bike

In [ ]:
# Create resource specification for an assembled bike
# TODO: here we pass a specification for the bike to be produced by the service provider
# when we will actually provide the bike, given that we also pass a design and a custom mirror,
# we will give another resource specification to the produced bike, and we will also use that 
# produce event as a satisfaction of the intent below. There will be no check that this might be
# an error, as we are practically overwriting what the intent was meant to produce.
name = 'assembled_bike'
note = 'Specification assembled bike'
classification = 'https://www.service_prov2.com/assembled/bike'
default_unit_id = units_data['piece']['id']
get_resource_spec_id(SPECS_FILE, users_data['service_prov2'], res_spec_data, name, note, classification, default_unit_id, endpoint=ENDPOINT)


In [ ]:
# Create the process that assembles the bike
process_name = 'Assemble_bike'
user_data = users_data['service_prov2']
note = f"Assemble bike 1-hour service by {user_data['name']}"
get_process(process_name, process_data, note, user_data, endpoint=ENDPOINT)

In [ ]:
# Create the proposal for encapsulating intent to assemble bike
name = "Bike_assembling_proposal"
note = "Proposal for assembling bike"
user_data = users_data['service_prov2']

hasBeginning = datetime.now(timezone.utc).isoformat()
dur_days = 365
hasEnd = (datetime.now(timezone.utc) + timedelta(days=dur_days)).isoformat()

eligibleLocation = locs_data['service_prov2']['id']

unitBased = True

get_proposal(name, proposal_data, note, user_data,\
             hasBeginning=hasBeginning, hasEnd=hasEnd, \
             eligibleLocation=eligibleLocation, unitBased=unitBased, endpoint=ENDPOINT)

In [ ]:
# Create the intent to produce a bike

name = "Bike_produce_intent"
note = "Intent to produce a bike"
user_data = users_data['service_prov2']

action = 'produce'
# References a concept in a common taxonomy or other classification scheme for purposes of categorization 
# or grouping.
resourceClassifiedAs = None
resourceConformsTo = res_spec_data['assembled_bike']['id']
resourceInventoriedAs = None
# Since we want to produce 1 bike we will put one here, the correct unity will be derived from the res specs
amount = 1
agreedIn = None
atLocation = locs_data['service_prov2']['id']
availableQuantity = None
effortQuantity = None
finished = False
due = None
hasBeginning = datetime.now(timezone.utc).isoformat()
dur_days = 365
hasEnd = (datetime.now(timezone.utc) + timedelta(days=dur_days)).isoformat()
hasPointInTime = None
inputOf = None
outputOf = process_data['Assemble_bike']['id']
provider = users_data['service_prov2']['id']
receiver = None

get_intent(name, intent_data, note, user_data, res_spec_data, provider=provider, receiver=receiver, action=action, \
           resourceClassifiedAs=resourceClassifiedAs, resourceConformsTo=resourceConformsTo, \
           resourceInventoriedAs=resourceInventoriedAs, amount=amount, agreedIn=agreedIn, \
           atLocation=atLocation, availableQuantity=availableQuantity, effortQuantity=effortQuantity,
           finished=finished, due=due, hasBeginning=hasBeginning, hasEnd=hasEnd, hasPointInTime=hasPointInTime,
           inputOf=inputOf, outputOf=outputOf, endpoint=ENDPOINT)


In [ ]:
# Create the proposed intent that publishes the intent in the proposal
name = 'Bike_assembling_publishing'
publishedIn = proposal_data['Bike_assembling_proposal']['id']
publishes = intent_data['Bike_produce_intent']['id']
reciprocal = False

get_proposedIntent(name, prop_int_data, user_data, publishedIn=publishedIn, publishes=publishes, \
                   reciprocal=reciprocal, endpoint=ENDPOINT)

# Implementation

## Create the design for the funky bike

In [ ]:
name = 'funky_bike_design'
note = 'Repository for funky bike design'
classification = 'https://github.com/designer1/funky_bike_design'
default_unit_id = units_data['piece']['id']
get_resource_spec_id(SPECS_FILE, users_data['designer1'], res_spec_data, name, note, classification, default_unit_id, endpoint=ENDPOINT)

In [ ]:
name = 'design_work'
note = 'Specification for design work'
classification = 'https://www.wikidata.org/wiki/Q82604'
default_unit_id = units_data['time']['id']
get_resource_spec_id(SPECS_FILE, users_data['designer1'], res_spec_data, name, note, classification, default_unit_id, endpoint=ENDPOINT)

In [ ]:
# Create the process that wraps creating design
process_name = 'Design_funky_bike'
user_data = users_data['designer1']
note = f"Design of funky bike by {user_data['name']}"
get_process(process_name, process_data, note, user_data, endpoint=ENDPOINT)

In [ ]:
# Define event work for the bike design
cur_res = action = event_note = amount = cur_pros = cur_int = None
action = 'work'
event_note='work to design funky bike'

cur_pros = process_data['Design_funky_bike']
effort_spec = {}
effort_spec['unit_id'] = res_spec_data['design_work']['defaultUnit']
effort_spec['spec_id'] = res_spec_data['design_work']['id']
effort_spec['amount'] = 8


event_id, ts = create_event(users_data['designer1'], action, event_note, amount=0, process=cur_pros, \
                 res_spec_data=res_spec_data, effort_spec=effort_spec, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'amount': effort_spec['amount']})
event_seq.append({'ts': ts, 'process_id':cur_pros['id'], 'name' : cur_pros['name']})

In [ ]:
# Define event produce for bike design
cur_res = action = event_note = amount = cur_pros = cur_int = None
action = 'produce'
event_note='produce design for funky bike'
amount = 1
cur_pros = process_data['Design_funky_bike']

res_data['funky_bike_design_res'] = {
    "res_ref_id": f'funky_bike_design-{random.randint(0, 10000)}',
    "name": 'funky bike design',
    "spec_id": res_spec_data['funky_bike_design']['id']
}
cur_res = res_data['funky_bike_design_res']


event_id, ts = create_event(users_data['designer1'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, new_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


## Manifacture bike frame

In [ ]:
# We create the resources that will not be saved to file as it is assumed they are recreated at each run
res_name = 'aluminium'
amount = 5
get_resource(res_data, res_spec_data, res_name, users_data['manufacturer1'], event_seq, amount, endpoint=ENDPOINT)

In [ ]:
# Define event work for the bike manufacturing
cur_res = action = event_note = amount = cur_pros = cur_int = None
action = 'work'
event_note='work to manufacture bike frame'

cur_pros = process_data['Manifacture_aluminium_bike_frame']
effort_spec = {}
effort_spec['unit_id'] = res_spec_data['manufacturing_work']['defaultUnit']
effort_spec['spec_id'] = res_spec_data['manufacturing_work']['id']
effort_spec['amount'] = 8


event_id, ts = create_event(users_data['manufacturer1'], action, event_note, amount=0, process=cur_pros, \
                 res_spec_data=res_spec_data, effort_spec=effort_spec, endpoint=ENDPOINT)

cur_int = intent_data['Bike_frame_work_intent']
name = 'bike_frame_work_sat'
note = f'satisfaction of {cur_int["name"]}'  
get_satisfaction(name, users_data['manufacturer1'], event_id, cur_int['id'], note, satisfaction_data, \
                 endpoint=ENDPOINT, effortQuantity=effort_spec)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'amount': effort_spec['amount']})

In [ ]:
# consume aluminium for bike frame
cur_res = action = event_note = amount = cur_pros = cur_int = None
action = 'consume'
event_note='consume aluminium for bike frame'
amount = 5
cur_pros = process_data['Manifacture_aluminium_bike_frame']
cur_res = res_data['aluminium_res']


event_id, ts = create_event(users_data['manufacturer1'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

cur_int = intent_data['Bike_frame_consume_intent']
name = 'bike_frame_consume_sat'
note = f'satisfaction of {cur_int["name"]}'  
get_satisfaction(name, users_data['manufacturer1'], event_id, cur_int['id'], note, satisfaction_data, \
                 endpoint=ENDPOINT, amount=amount, cur_res=cur_res, res_spec_data=res_spec_data)


event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})
event_seq.append({'ts': ts, 'process_id':cur_pros['id'], 'name' : cur_pros['name']})


In [ ]:
# Define event produce for aluminium bike frame
cur_res = action = event_note = amount = cur_pros = cur_int = None
action = 'produce'
event_note='produce aluminium bike frame'
amount = 1
cur_pros = process_data['Manifacture_aluminium_bike_frame']

res_data['aluminium_bike_frame_res'] = {
    "res_ref_id": f'aluminium_bike_frame-{random.randint(0, 10000)}',
    "name": 'aluminium bike frame',
    "spec_id": res_spec_data['aluminium_bike_frame']['id']
}
cur_res = res_data['aluminium_bike_frame_res']


event_id, ts = create_event(users_data['manufacturer1'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, new_res=cur_res, endpoint=ENDPOINT)

cur_int = intent_data['Bike_frame_produce_intent']
name = 'bike_frame_produce_sat'
note = f'satisfaction of {cur_int["name"]}'  
get_satisfaction(name, users_data['manufacturer1'], event_id, cur_int['id'], note, satisfaction_data, \
                 endpoint=ENDPOINT, amount=amount, cur_res=cur_res, res_spec_data=res_spec_data)


event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# Transfer the aluminium bike frame from manufacturer1 to designer2
cur_res = action = event_note = amount = cur_pros = cur_int = None
note='Transfer aluminium bike frame from manufacturer1 to designer2'
action = 'transfer'
amount = 1
cur_res = res_data['aluminium_bike_frame_res']

event_id, ts = make_transfer(users_data['manufacturer1'], action, note, users_data['designer2'], amount, cur_res, locs_data, res_spec_data, endpoint=ENDPOINT)
event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


## Create the design for the magic mirror

In [ ]:
name = 'magic_mirror_design'
note = 'Repository for magic mirror design'
classification = 'https://github.com/designer2/magic_mirror_design'
default_unit_id = units_data['piece']['id']
get_resource_spec_id(SPECS_FILE, users_data['designer2'], res_spec_data, name, note, classification, default_unit_id, endpoint=ENDPOINT)

In [ ]:
# Create the process that wraps creating design
process_name = 'Design_magic_mirror'
user_data = users_data['designer2']
note = f"Design of magic mirror by {user_data['name']}"
get_process(process_name, process_data, note, user_data, endpoint=ENDPOINT)

In [ ]:
# Define event work for magic mirror design
cur_res = action = event_note = amount = cur_pros = cur_int = None
action = 'work'
event_note='work to design magic mirror'

cur_pros = process_data['Design_magic_mirror']
effort_spec = {}
effort_spec['unit_id'] = res_spec_data['design_work']['defaultUnit']
effort_spec['spec_id'] = res_spec_data['design_work']['id']
effort_spec['amount'] = 5


event_id, ts = create_event(users_data['designer2'], action, event_note, amount=0, process=cur_pros, \
                 res_spec_data=res_spec_data, effort_spec=effort_spec, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'amount': effort_spec['amount']})
event_seq.append({'ts': ts, 'process_id':cur_pros['id'], 'name' : cur_pros['name']})

In [ ]:
# Define event produce for magic mirror design
cur_res = action = event_note = amount = cur_pros = cur_int = None
action = 'produce'
event_note='produce design for magic mirror'
amount = 1
cur_pros = process_data['Design_magic_mirror']

res_data['magic_mirror_design_res'] = {
    "res_ref_id": f'magic_mirror_design-{random.randint(0, 10000)}',
    "name": 'magic mirror design',
    "spec_id": res_spec_data['magic_mirror_design']['id']
}
cur_res = res_data['magic_mirror_design_res']


event_id, ts = create_event(users_data['designer2'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, new_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


## Create Magic Mirror

In [ ]:
# Resource specification for glass
name = 'glass'
note = 'Glass for magic bike mirror'
classification = 'https://www.wikidata.org/wiki/Q11469'
default_unit_id = units_data['mass']['id']
get_resource_spec_id(SPECS_FILE, users_data['manufacturer2'], res_spec_data, name, note, classification, default_unit_id, endpoint=ENDPOINT)

In [ ]:
res_name = 'glass'
amount = .2
get_resource(res_data, res_spec_data, res_name, users_data['manufacturer2'], event_seq, amount, endpoint=ENDPOINT)

In [ ]:
# Create the process that wraps creating the mirror
process_name = 'Creation_magic_bike_mirror'
user_data = users_data['manufacturer2']
note = f"Creation of magic bike mirror by {user_data['name']}"
get_process(process_name, process_data, note, user_data, endpoint=ENDPOINT)


In [ ]:
# Create the resource specification for magic bike mirror
name = 'magic_bike_mirror'
note = 'Repository for magic bike mirror'
classification = 'https://github.com/manufacturer2/magic_bike_mirror'
default_unit_id = units_data['piece']['id']
get_resource_spec_id(SPECS_FILE, users_data['manufacturer2'], res_spec_data, name, note, classification, default_unit_id, endpoint=ENDPOINT)

In [ ]:
# cite design for bike
cur_res = action = event_note = amount = cur_pros = cur_int = None
action = 'cite'
event_note='cite design to make mirror'
amount = 1
cur_pros = process_data['Creation_magic_bike_mirror']
cur_res = res_data['magic_mirror_design_res']


event_id, ts = create_event(users_data['manufacturer2'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# consume glass for mirror
cur_res = action = event_note = amount = cur_pros = cur_int = None
action = 'consume'
event_note='consume glass for bike mirror'
amount = .2
cur_pros = process_data['Creation_magic_bike_mirror']
cur_res = res_data['glass_res']


event_id, ts = create_event(users_data['manufacturer2'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})
event_seq.append({'ts': ts, 'process_id':cur_pros['id'], 'name' : cur_pros['name']})


In [ ]:
# Define event produce for bike mirror
cur_res = action = event_note = amount = cur_pros = cur_int = None
action = 'produce'
event_note='produce magic bike mirror'
amount = 1
cur_pros = process_data['Creation_magic_bike_mirror']

res_data['magic_bike_mirror_res'] = {
    "res_ref_id": f'magic_bike_mirror-{random.randint(0, 10000)}',
    "name": 'magic bike mirror',
    "spec_id": res_spec_data['magic_bike_mirror']['id']
}
cur_res = res_data['magic_bike_mirror_res']


event_id, ts = create_event(users_data['manufacturer2'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, new_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


## Paint the mirror

In [ ]:
name = 'painting_work'
note = 'Specification for painting work'
classification = 'https://www.wikidata.org/wiki/Q11629'
default_unit_id = units_data['time']['id']
get_resource_spec_id(SPECS_FILE, users_data['manufacturer2'], res_spec_data, name, note, classification, default_unit_id, endpoint=ENDPOINT)

In [ ]:
# Create the process that wraps painting the mirror
process_name = 'Painting_magic_bike_mirror'
user_data = users_data['manufacturer2']
note = f"Painting magic bike mirror by {user_data['name']}"
get_process(process_name, process_data, note, user_data, endpoint=ENDPOINT)


In [ ]:
# Define event work for magic mirror painting
cur_res = action = event_note = amount = cur_pros = cur_int = None
action = 'work'
event_note='work to paint magic mirror'

cur_pros = process_data['Painting_magic_bike_mirror']
effort_spec = {}
effort_spec['unit_id'] = res_spec_data['painting_work']['defaultUnit']
effort_spec['spec_id'] = res_spec_data['painting_work']['id']
effort_spec['amount'] = 6


event_id, ts = create_event(users_data['manufacturer2'], action, event_note, amount=0, process=cur_pros, \
                 res_spec_data=res_spec_data, effort_spec=effort_spec, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'amount': effort_spec['amount']})

In [ ]:
# accept mirror for painting
cur_res = action = event_note = amount = cur_pros = cur_int = None
action = 'accept'
event_note='accept mirror for painting'
amount = 1
cur_pros = process_data['Painting_magic_bike_mirror']
cur_res = res_data['magic_bike_mirror_res']


event_id, ts = create_event(users_data['manufacturer2'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})
event_seq.append({'ts': ts, 'process_id':cur_pros['id'], 'name' : cur_pros['name']})


In [ ]:
# modify the mirrow that is now painted
cur_res = action = event_note = amount = cur_pros = None
action = 'modify'
event_note='modify mirror into painted mirror'
amount = 1
cur_pros = process_data['Painting_magic_bike_mirror']
cur_res = res_data['magic_bike_mirror_res']


event_id, ts = create_event(users_data['manufacturer2'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})



## Package the mirror

In [ ]:
name = 'packaging_work'
note = 'Specification for packaging work'
classification = 'https://www.wikidata.org/wiki/Q207822'
default_unit_id = units_data['time']['id']
get_resource_spec_id(SPECS_FILE, users_data['manufacturer2'], res_spec_data, name, note, classification, default_unit_id, endpoint=ENDPOINT)

In [ ]:
# Create the process that wraps packaging the mirror
process_name = 'Packaging_magic_bike_mirror'
user_data = users_data['manufacturer2']
note = f"Packaging magic bike mirror by {user_data['name']}"
get_process(process_name, process_data, note, user_data, endpoint=ENDPOINT)


In [ ]:
# Define event work for magic mirror painting
cur_res = action = event_note = amount = cur_pros = cur_int = None
action = 'work'
event_note='work to package magic mirror'

cur_pros = process_data['Packaging_magic_bike_mirror']
effort_spec = {}
effort_spec['unit_id'] = res_spec_data['packaging_work']['defaultUnit']
effort_spec['spec_id'] = res_spec_data['packaging_work']['id']
effort_spec['amount'] = 3


event_id, ts = create_event(users_data['manufacturer2'], action, event_note, amount=0, process=cur_pros, \
                 res_spec_data=res_spec_data, effort_spec=effort_spec, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'amount': effort_spec['amount']})

In [ ]:
# accept mirror for packaging
cur_res = action = event_note = amount = cur_pros = cur_int = None
action = 'accept'
event_note='accept mirror for packaging'
amount = 1
cur_pros = process_data['Packaging_magic_bike_mirror']
cur_res = res_data['magic_bike_mirror_res']


event_id, ts = create_event(users_data['manufacturer2'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})
event_seq.append({'ts': ts, 'process_id':cur_pros['id'], 'name' : cur_pros['name']})


In [ ]:
# modify the mirrow that is now packaged
cur_res = action = event_note = amount = cur_pros = None
action = 'modify'
event_note='modify mirror into packaged mirror'
amount = 1
cur_pros = process_data['Packaging_magic_bike_mirror']
cur_res = res_data['magic_bike_mirror_res']


event_id, ts = create_event(users_data['manufacturer2'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})



In [ ]:
# Transfer the magic bike mirror from manufacturer2 to designer2
cur_res = action = event_note = amount = cur_pros = cur_int = None
note='Transfer magic bike mirror from manufacturer2 to designer2'
action = 'transfer'
amount = 1
cur_res = res_data['magic_bike_mirror_res']

event_id, ts = make_transfer(users_data['manufacturer2'], action, note, users_data['designer2'], amount, cur_res, locs_data, res_spec_data, endpoint=ENDPOINT)
event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


## Assemble the bike

In [ ]:
# Transfer the magic bike mirror from designer2 to service_prov2
cur_res = action = event_note = amount = cur_pros = cur_int = None
note='Transfer magic bike mirror from designer2 to service_prov2'
action = 'transfer'
amount = 1
cur_res = res_data['magic_bike_mirror_res']

event_id, ts = make_transfer(users_data['designer2'], action, note, users_data['service_prov2'], amount, cur_res, locs_data, res_spec_data, endpoint=ENDPOINT)
event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# Transfer the aluminium bike frame from designer2 to service_prov2
cur_res = action = event_note = amount = cur_pros = cur_int = None
note='Transfer aluminium bike frame from designer2 to service_prov2'
action = 'transfer'
amount = 1
cur_res = res_data['aluminium_bike_frame_res']

event_id, ts = make_transfer(users_data['designer2'], action, note, users_data['service_prov2'], amount, cur_res, locs_data, res_spec_data, endpoint=ENDPOINT)
event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# TODO: This process is not needed in theory as we already defined one in the intent
# But is it wrong to do so? Should it be checked somehow?

# Create the process that wraps creating the bike
# process_name = 'Creation_fancy_collaborative_bike'
# user_data = users_data['designer2']
# note = f"Creation of fancy collaborative bike by {user_data['name']}"
# get_process(process_name, process_data, note, user_data, endpoint=ENDPOINT)


In [ ]:
# consume mirror for bike
cur_res = action = event_note = amount = cur_pros = cur_int = None
action = 'consume'
event_note='consume mirror for bike'
amount = 1
cur_pros = process_data['Assemble_bike']
cur_res = res_data['magic_bike_mirror_res']


event_id, ts = create_event(users_data['service_prov2'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# cite design for bike
cur_res = action = event_note = amount = cur_pros = cur_int = None
action = 'cite'
event_note='cite design to build bike'
amount = 1
cur_pros = process_data['Assemble_bike']
cur_res = res_data['funky_bike_design_res']


event_id, ts = create_event(users_data['service_prov2'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# consume aluminium bike frame for bike
cur_res = action = event_note = amount = cur_pros = cur_int = None
action = 'consume'
event_note='consume aluminium bike frame'
amount = 1
cur_pros = process_data['Assemble_bike']
cur_res = res_data['aluminium_bike_frame_res']


event_id, ts = create_event(users_data['service_prov2'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
name = 'assembleing_work'
note = 'Specification for assembleing work'
classification = 'https://www.wikidata.org/wiki/Q187939'
default_unit_id = units_data['time']['id']
get_resource_spec_id(SPECS_FILE, users_data['service_prov2'], res_spec_data, name, note, classification, default_unit_id, endpoint=ENDPOINT)


In [ ]:
# Define event work for the bike assembling
cur_res = action = event_note = amount = cur_pros = cur_int = None
action = 'work'
event_note='work to assemble bike'

cur_pros = process_data['Assemble_bike']
effort_spec = {}
effort_spec['unit_id'] = res_spec_data['assembleing_work']['defaultUnit']
effort_spec['spec_id'] = res_spec_data['assembleing_work']['id']
effort_spec['amount'] = 1


event_id, ts = create_event(users_data['service_prov2'], action, event_note, amount=0, process=cur_pros, \
                 res_spec_data=res_spec_data, effort_spec=effort_spec, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'amount': effort_spec['amount']})
event_seq.append({'ts': ts, 'process_id':cur_pros['id'], 'name' : cur_pros['name']})

In [ ]:
# Resource specification for the bike
name = 'fancy_collaborative_bike'
note = 'Repository for fancy collaborative bike'
classification = 'https://github.com/designer2/fancy_collaborative_bike'
default_unit_id = units_data['piece']['id']
get_resource_spec_id(SPECS_FILE, users_data['designer2'], res_spec_data, name, note, classification, default_unit_id, endpoint=ENDPOINT)


In [ ]:
# Define event produce for collaborative bike
cur_res = action = event_note = amount = cur_pros = cur_int = None
action = 'produce'
event_note='produce fancy collaborative bike'
amount = 1
cur_pros = process_data['Assemble_bike']

res_data['fancy_collaborative_bike_res'] = {
    "res_ref_id": f'fancy_collaborative_bike-{random.randint(0, 10000)}',
    "name": 'fancy collaborative bike',
    "spec_id": res_spec_data['fancy_collaborative_bike']['id']
}
cur_res = res_data['fancy_collaborative_bike_res']


event_id, ts = create_event(users_data['service_prov2'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, new_res=cur_res, endpoint=ENDPOINT)

cur_int = intent_data['Bike_produce_intent']
name = 'bike_produce_sat'
note = f'satisfaction of {cur_int["name"]}'  

get_satisfaction(name, users_data['service_prov2'], event_id, cur_int['id'], note, satisfaction_data, \
                 endpoint=ENDPOINT, amount=amount, cur_res=cur_res, res_spec_data=res_spec_data)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# Transfer the fancy bike from service_prov2 to designer2
cur_res = action = event_note = amount = cur_pros = cur_int = None
note='Transfer fancy bike from service_prov2 to designer2'
action = 'transfer'
amount = 1
cur_res = res_data['fancy_collaborative_bike_res']

event_id, ts = make_transfer(users_data['service_prov2'], action, note, users_data['designer2'], amount, cur_res, locs_data, res_spec_data, endpoint=ENDPOINT)
event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
show_proposal(users_data['designer2'], proposal_data['Bike_assembling_proposal']['id'], endpoint=ENDPOINT)

In [ ]:
check_proposals(users_data['designer2'], proposal_data, endpoint=ENDPOINT)

## Groups processes

In [ ]:
# Create the process group to hold the processes
name = note = processgrp = process = processgrp_id = None
name = "design_assemble_bike"
note = "Groups bike design and assemblage"

get_processgrp(name, users_data['designer2'], note, processgrp_data, processgrp_id=processgrp_id, endpoint=ENDPOINT)


In [ ]:
# Insert the processes in the group
name = note = processgrp = process = processgrp_id = None

processgrp = processgrp_data["design_assemble_bike"]
process = process_data['Design_funky_bike']

insert_procingrp(users_data['designer2'], processgrp, process, endpoint=ENDPOINT)

process = process_data['Assemble_bike']

insert_procingrp(users_data['designer2'], processgrp, process, endpoint=ENDPOINT)

In [ ]:
# Create the process group to hold two process groups
name = note = processgrp = process = processgrp_id = None
name = "design_create_paint_package_mirror"
note = "Groups two groups: mirror design and creation and mirror painting and packaging"

get_processgrp(name, users_data['designer2'], note, processgrp_data, processgrp_id=processgrp_id, endpoint=ENDPOINT)

In [ ]:
# Create the process group to hold the processes
name = note = processgrp = process = processgrp_id = None
name = "design_create_mirror"
note = "Groups mirror design and creation"
processgrp_id = processgrp_data['design_create_paint_package_mirror']['id']

get_processgrp(name, users_data['designer2'], note, processgrp_data, processgrp_id=processgrp_id, endpoint=ENDPOINT)


In [ ]:
# Insert the processes in the group
name = note = processgrp = process = processgrp_id = None

processgrp = processgrp_data["design_create_mirror"]
process = process_data['Design_magic_mirror']

insert_procingrp(users_data['designer2'], processgrp, process, endpoint=ENDPOINT)

process = process_data['Creation_magic_bike_mirror']

insert_procingrp(users_data['designer2'], processgrp, process, endpoint=ENDPOINT)

In [ ]:
# Create the process group to hold the processes
name = note = processgrp = process = processgrp_id = None
name = "paint_package_mirror"
note = "Groups mirror painting and packaging"
processgrp_id = processgrp_data['design_create_paint_package_mirror']['id']

get_processgrp(name, users_data['designer2'], note, processgrp_data, processgrp_id=processgrp_id, endpoint=ENDPOINT)

In [ ]:
# Insert the processes in the group
name = note = processgrp = process = processgrp_id = None

processgrp = processgrp_data["paint_package_mirror"]
process = process_data['Painting_magic_bike_mirror']

insert_procingrp(users_data['designer2'], processgrp, process, endpoint=ENDPOINT)

process = process_data['Packaging_magic_bike_mirror']

insert_procingrp(users_data['designer2'], processgrp, process, endpoint=ENDPOINT)

## Start tracing

In [ ]:
show_data(users_data, locs_data, res_data, units_data, res_spec_data, process_data, event_seq, \
          proposal_data, intent_data, prop_int_data, satisfaction_data, processgrp_data)

In [ ]:
trace_me = res_data['fancy_collaborative_bike_res']['id']
print(f"Resource to be traced: {trace_me}")
tot_dpp = []
visited = set()
er_before(trace_me, users_data['designer2'], dpp_children=tot_dpp, depth=0, visited=visited, endpoint=ENDPOINT)

# Serializing json
json_object = json.dumps(tot_dpp, indent=2)

print(json_object)
print(visited)

In [ ]:
be_dpp = get_dpp(trace_me, users_data['designer2'], endpoint=ENDPOINT)
print(json.dumps(be_dpp, indent=2))

In [ ]:
trace = trace_query(trace_me, users_data['designer2'], endpoint=ENDPOINT)

In [ ]:
check_traces(trace, event_seq, tot_dpp, be_dpp)
save_traces(USE_CASE, tot_dpp, trace, be_dpp, event_seq, processgrp_data)

In [ ]:
print(json.dumps(processgrp_data, indent=2))

In [ ]:
labels = []
sources = []
targets = []
values = []
color_nodes = []
color_links = []
assigned = {}
vis_dpp(tot_dpp[0], count=0, assigned=assigned, labels=labels, targets=targets, sources=sources, values=values, color_nodes=color_nodes, color_links=color_links)
sources, targets = consol_trace(assigned, sources, targets)
make_sankey(sources, targets, labels, values, color_nodes, color_links)

In [ ]:
# STOP HERE!!